In [58]:
from pymongo import MongoClient
import pandas as pd
import requests
import json

## 1. Clean data about best markets to locate a gaming company

In [59]:
games_video_raw_df = pd.read_csv('../data/video_games_companies.csv')

In [60]:
games_video_raw_df

,name,category_code,number_of_employees,total_money_raised,country_code,city,address,zip_code,latitude,longitude
0,Flektor,games_video,NaN,$0,USA,Culver City,"8536 National Blvd, Suite A",90232,34.025958,-118.379768
1,Lala,games_video,NaN,$44.2M,USA,Palo Alto,209 Hamilton Ave,94301,37.451151,-122.154369
2,Joost,games_video,0.0,$45M,USA,New York,100 5th Ave Fl 6,10011-6903,40.746497,-74.009447
3,Babelgum,games_video,NaN,$13.2M,GBR,London,NaN,NaN,53.344104,-6.267494
4,Sparter,games_video,NaN,$0,USA,NaN,NaN,NaN,37.090240,-95.712891
...,...,...,...,...,...,...,...,...,...,...
1078,Symposium Productions,games_video,NaN,$0,NaN,NaN,NaN,NaN,NaN,NaN
1079,Rev Media,games_video,NaN,$0,USA,Dallas,NaN,NaN,NaN,NaN
1080,Fictionaut,games_video,2.0,$0,USA,Astoria,32-14 42nd St,11103,40.759795,-73.916127
1081,Tiny Speck,games_video,45.0,$17.2M,CAN,Vancouver,163 Hastings St W,BC V6B 1H5,49.282455,-123.109217


### 1.1 I want to see null country values in case I can handdle them if I have any information that can point out where the company is located. Otherwise, I will drop this rows.

In [61]:
null_countries_count = games_video_raw_df['country_code'].isnull().sum()
null_countries_count

244

In [62]:
# Drop rows where all specific columns are null at the same time
games_video_cleaned_df = games_video_raw_df.dropna(subset=['country_code', 'city', 'latitude', 'address','longitude', 'zip_code'], how='all')

# Display the shape of the DataFrame before and after the new cleaning to see how many rows were dropped
games_video_cleaned_shape = games_video_cleaned_df.shape

print(f'Shape before: {games_video_raw_df.shape}')
print(f'Shape after: {games_video_cleaned_df.shape}')

Shape before: (1083, 10)
Shape after: (839, 10)


In [63]:
games_video_cleaned_df.to_csv('../data/video_games_companies_cleaned.csv', index=False)

## 2. Clean data about possible offices in the USA

In [64]:
usa_companies_raw_df = pd.read_csv('../data/usa_companies.csv')

In [65]:
usa_companies_raw_df

,name,category_code,number_of_employees,total_money_raised,country_code,city,address,zip_code,latitude,longitude
0,Wetpaint,web,47.0,$39.8M,USA,Seattle,710 - 2nd Avenue,98104,47.603122,-122.333253
1,AdventNet,enterprise,600.0,$0,USA,Pleasanton,4900 Hopyard Rd.,94588,37.692934,-121.904945
2,Zoho,software,1600.0,$0,USA,Pleasanton,4900 Hopyard Rd,94588,37.692934,-121.904945
3,Digg,news,60.0,$45M,USA,San Francisco,135 Mississippi St,94107,37.764726,-122.394523
4,Facebook,social,5299.0,$2.43B,USA,Menlo Park,1601 Willow Road,94025,37.416050,-122.151801
...,...,...,...,...,...,...,...,...,...,...
9261,Contemporary Computer Services,software,NaN,$0,USA,Bohemia,200 Knickerbocker Avenue,11716,40.775055,-73.088140
9262,QSGI,software,164.0,$0,USA,Palm Beach,400 Royal Palm Way,33480,26.705331,-80.041395
9263,AfterLogic,software,NaN,$0,USA,Livingston,P.O. Box 2096,07039,40.793024,-74.323554
9264,EnteGreat Solutions,software,NaN,$0,USA,Birmingham,NaN,NaN,33.518885,-86.816068


### 2.1 I want to drop all rows that have null values in the columns address, latitude and longitude at the same time

In [66]:
# Drop rows where all specific columns are null at the same time
usa_companies_cleaned_df = usa_companies_raw_df.dropna(subset=['latitude', 'address','longitude', 'zip_code'], how='all')

# Display the shape of the DataFrame before and after the new cleaning to see how many rows were dropped
usa_companies_cleaned_shape = usa_companies_cleaned_df.shape

print(f'Shape before: {usa_companies_raw_df.shape}')
print(f'Shape after: {usa_companies_cleaned_df.shape}')

Shape before: (9266, 10)
Shape after: (9055, 10)


In [67]:
usa_companies_cleaned_df.to_csv('../data/usa_companies_cleaned.csv', index=False)

## 3. Clean Design Companies and Companies that have raised more than 1M dollars

In [68]:
required_companies_raw_df = pd.read_csv('../data/required_companies.csv')

In [69]:
required_companies_raw_df

,name,category_code,total_money_raised,country_code,city,address,zip_code,latitude,longitude
0,Digg,news,$45M,USA,San Francisco,135 Mississippi St,94107,37.764726,-122.394523
1,Joost,games_video,$45M,USA,New York,100 5th Ave Fl 6,10011-6903,40.746497,-74.009447
2,SpinVox,messaging,$106M,GBR,Buckinghamshire,"Wethered House, Pound Lane",SL7 2AF,NaN,NaN
3,AddThis,advertising,$73M,USA,Vienna,1595 Spring Hill Road,22182,38.926172,-77.245195
4,Mahalo,web,$21M,USA,Culver City,3525 Eastham Dr.,90232,34.017606,-118.487267
...,...,...,...,...,...,...,...,...,...
1651,Luminus Devices,other,$136M,USA,Billerica,1100 Technology Park Drive,01821,42.528635,-71.278022
1652,Cameron Health,medical,$121M,USA,San Clemente,905 Calle Amanecer,92673,33.448010,-117.607717
1653,Cantimer,NaN,$2M,USA,Menlo Park,3700 Haven Court,94025,37.484616,-122.181753
1654,Celestial Semiconductor,semiconductor,$2M,USA,San Jose,"2635 North 1st Street, Suite 150",95134,37.386417,-121.929170


### 3.1 I want companies only in San Francisco and New York

In [70]:
# Define the cities of interest for the new requirement
cities_of_interest_new = ['San Francisco', 'New York']

# Filter the dataframe for companies in San Francisco and New York
required_companies_cleaned_df = required_companies_raw_df[
    required_companies_raw_df['city'].isin(cities_of_interest_new)]

required_companies_cleaned_df.reset_index(drop=True, inplace=True)
required_companies_cleaned_df


,name,category_code,total_money_raised,country_code,city,address,zip_code,latitude,longitude
0,Digg,news,$45M,USA,San Francisco,135 Mississippi St,94107,37.764726,-122.394523
1,Joost,games_video,$45M,USA,New York,100 5th Ave Fl 6,10011-6903,40.746497,-74.009447
2,Prosper,finance,$120M,USA,San Francisco,101 2nd Street,94105,37.789760,-122.402524
3,Pando Networks,games_video,$11M,USA,New York,520 Broadway,10012,40.722655,-73.998730
4,Revision3,games_video,$9M,USA,San Francisco,2415 3rd St,94107,37.757758,-122.388243
...,...,...,...,...,...,...,...,...,...
256,Teliris,public_relations,$11M,USA,New York,"100 William Street, Suite 1825",10038,42.902965,-78.883572
257,WiNetworks,mobile,$19M,USA,New York,"555 West 57th Street, Suite 1326",10019,40.770437,-73.990426
258,TeachScape,software,$16M,USA,San Francisco,71 Stevenson St.,94105,37.786905,-122.404392
259,Roc2Loc,cleantech,$2M,USA,San Francisco,"66 Mint Street, 3rd Floor",94103,37.782475,-122.407764


In [71]:
required_companies_cleaned_df.to_csv('../data/required_companies_cleaned.csv', index=False)

## 4. Merging companies of interest

In [84]:
# Load the CSV files into DataFrames
pre_selected_df = pd.read_csv('../data/pre_selected_companies.csv')
required_companies_selected_df = pd.read_csv('../data/required_companies_cleaned.csv')

# Add an identifier column to the pre-selected DataFrame
pre_selected_df['Company_Type'] = 'Pre-selected'

# Assuming there's a column 'total_money_raised' to check if a company raised more than $1M
# And a column 'category_code' where 'design' might be a category for design companies
# Modify the lambda function to interpret strings with 'M'
required_companies_selected_df['Company_Type'] = required_companies_selected_df.apply(
    lambda x: 'Design' if x['category_code'] == 'design' 
    else ('Raised > $1M' if 'M' in str(x['total_money_raised']) else 'Other'),
    axis=1
)

# Concatenate the DataFrames
all_companies_df = pd.concat([pre_selected_df, required_companies_selected_df], ignore_index=True)

# If you need to remove duplicates
#all_companies_df.drop_duplicates(subset=['Company_Identifier_Column'], inplace=True)

# Inspect the combined DataFrame
all_companies_df


,name,category_code,number_of_employees,total_money_raised,country_code,city,address,zip_code,latitude,longitude,Company_Type
0,Livestream,games_video,120.0,$14.7M,USA,New York,"111 8th Avenue, #1509",10011,40.726155,-73.995625,Pre-selected
1,hi5,games_video,100.0,$52M,USA,San Francisco,55 Second Street,94105,37.788668,-122.400558,Pre-selected
2,Zynga,games_video,115.0,$860M,USA,San Francisco,365 Vermont St.,94103,37.765158,-122.404234,Pre-selected
3,iWin,games_video,103.0,$0,USA,San Francisco,45 Fremont Street,94105,37.781689,-122.391061,Pre-selected
4,Exent,games_video,100.0,$3M,USA,New York,632 W. 28th St.,10001,40.752380,-74.005568,Pre-selected
...,...,...,...,...,...,...,...,...,...,...,...
261,Teliris,public_relations,NaN,$11M,USA,New York,"100 William Street, Suite 1825",10038,42.902965,-78.883572,Raised > $1M
262,WiNetworks,mobile,NaN,$19M,USA,New York,"555 West 57th Street, Suite 1326",10019,40.770437,-73.990426,Raised > $1M
263,TeachScape,software,NaN,$16M,USA,San Francisco,71 Stevenson St.,94105,37.786905,-122.404392,Raised > $1M
264,Roc2Loc,cleantech,NaN,$2M,USA,San Francisco,"66 Mint Street, 3rd Floor",94103,37.782475,-122.407764,Raised > $1M


In [86]:
# Drop duplicates
all_companies_cleaned_df = all_companies_df.drop_duplicates()
all_companies_cleaned_df

,name,category_code,number_of_employees,total_money_raised,country_code,city,address,zip_code,latitude,longitude,Company_Type
0,Livestream,games_video,120.0,$14.7M,USA,New York,"111 8th Avenue, #1509",10011,40.726155,-73.995625,Pre-selected
1,hi5,games_video,100.0,$52M,USA,San Francisco,55 Second Street,94105,37.788668,-122.400558,Pre-selected
2,Zynga,games_video,115.0,$860M,USA,San Francisco,365 Vermont St.,94103,37.765158,-122.404234,Pre-selected
3,iWin,games_video,103.0,$0,USA,San Francisco,45 Fremont Street,94105,37.781689,-122.391061,Pre-selected
4,Exent,games_video,100.0,$3M,USA,New York,632 W. 28th St.,10001,40.752380,-74.005568,Pre-selected
...,...,...,...,...,...,...,...,...,...,...,...
261,Teliris,public_relations,NaN,$11M,USA,New York,"100 William Street, Suite 1825",10038,42.902965,-78.883572,Raised > $1M
262,WiNetworks,mobile,NaN,$19M,USA,New York,"555 West 57th Street, Suite 1326",10019,40.770437,-73.990426,Raised > $1M
263,TeachScape,software,NaN,$16M,USA,San Francisco,71 Stevenson St.,94105,37.786905,-122.404392,Raised > $1M
264,Roc2Loc,cleantech,NaN,$2M,USA,San Francisco,"66 Mint Street, 3rd Floor",94103,37.782475,-122.407764,Raised > $1M


In [87]:
all_companies_cleaned_df.to_csv('../data/all_companies_cleaned_df.csv', index=False)

## 5. Cleaning Latitude and Longitude from the Final Dataframe 

In [144]:
# Create a copy of the DataFrame to avoid SettingWithCopyWarning
all_companies_cleaned_df = all_companies_cleaned_df.copy()

# Define the function to convert and handle NA values
def convert_to_float(value):
    if pd.isna(value):
        return pd.NA
    elif value == "Throttled! See geocode.xyz/pricing":
        return pd.NA
    else:
        try:
            return float(value)
        except ValueError:
            return pd.NA

# Apply the function to the 'latitude' and 'longitude' columns
all_companies_cleaned_df['latitude'] = all_companies_cleaned_df['latitude'].apply(convert_to_float)
all_companies_cleaned_df['longitude'] = all_companies_cleaned_df['longitude'].apply(convert_to_float)

In [146]:
# Drop rows with "Unknown" in 'address' column
all_companies_cleaned_df = all_companies_cleaned_df[all_companies_cleaned_df['address'] != "Unknown"]

# Drop rows with NaN in 'latitude' or 'longitude' columns
all_companies_cleaned = all_companies_cleaned_df.dropna(subset=['latitude', 'longitude'])

# Check the first few rows to verify the changes
all_companies_cleaned_df.to_csv('../data/all_companies_cleaned_df.csv', index=False)

In [126]:
from getpass import getpass
import time
import os
from dotenv import load_dotenv
import numpy as np

In [93]:
load_dotenv()  # Take environment variables from .env.

True

In [106]:
token = os.getenv("token") 

In [152]:
MAX_ATTEMPTS = 3

def gets_coordinates_from_string (address, attemps):
    if attemps > MAX_ATTEMPTS:
        return None
    
    url_geocode = f"https://geocode.xyz/{address}?json=1"
    res = requests.get(url_geocode)
    
    try:
        response = res.json()
        
        # The API can fail in two ways, it either didn't find the address or the call was throttled
        if 'latt' in response and 'Throttled' not in response['latt']:
            return response
    except ex:
        print(f"Couldn't get geo data for {address}, attempts: {attempts}, error: {str(ex)}")
        
    # If we get here we haven't found geo data yet
    time.sleep(10)
    return gets_coordinates_from_string(address,attemps+1)

In [153]:
row = {'address': '201 Third Street, 2nd Floor', 'city': 'San Francisco'}
full_address = f"{row['address']}, {row['city']}"
test1 = gets_coordinates_from_string(full_address, 0)
test1

{'standard': {'stnumber': '4732',
  'addresst': 'Third Street',
  'statename': 'California',
  'postal': {},
  'region': 'CA',
  'prov': 'US',
  'city': 'San Francisco',
  'countryname': 'United States of America',
  'confidence': '0.9'},
 'longt': '-122.39078',
 'alt': {},
 'elevation': {},
 'latt': '37.73518'}

In [158]:
# First, filter out rows where latitude and longitude are missing
#companies_to_geocode = all_companies_cleaned_df[all_companies_cleaned_df['latitude'].isnull() | all_companies_cleaned_df['longitude'].isnull()]

# Then loop through these companies and geocode them
for index, row in all_companies_cleaned_df.iterrows():    
    row_lat = all_companies_cleaned_df.at[index, 'latitude']
    row_lng = all_companies_cleaned_df.at[index, 'longitude']
    address = all_companies_cleaned_df.at[index, 'address']
    
    lat_or_lng_empty = pd.isna(row_lat) and pd.isna(row_lng)
    has_address = type(address) == str
    
    # We only add information to rows that don't have lat and lng, that also have an address for search
    if lat_or_lng_empty and has_address:
        city = all_companies_cleaned_df.at[index, 'city']
        
        # We concat address and city to increase reliability
        full_address = f"{address}, {city}"
        print(f"Getting the geo data for {full_address}")
        
        response = gets_coordinates_from_string(full_address, 0)
        
        if response is None:
            print(f"Couldn't get geo data for address {full_address}")
            continue
        
        api_lat = float(response['latt'])
        api_lng = float(response['longt'])

        # Update the DataFrame with the new latitude and longitude
        all_companies_cleaned_df.at[index, 'latitude'] = api_lat
        all_companies_cleaned_df.at[index, 'longitude'] = api_lng

        # API rate limit
        time.sleep(5)

# Save the updated DataFrame
all_companies_cleaned_df.to_csv('../data/all_companies_cleaned_df.csv', index=False)


40.726155 -73.995625
37.788668 -122.400558
37.765158 -122.404234
37.781689 -122.391061
40.7523796 -74.005568
37.764726 -122.394523
40.7464969 -74.0094471
37.78976 -122.402524
40.722655 -73.99873
37.757758 -122.388243
37.786183 -122.402195
40.7516611 -73.9843328
37.789268 -122.395184
37.781427 -122.392144
37.782103 -122.401116
37.762681 -122.400909
37.779507 -122.39071
37.775196 -122.419204
37.793703 -122.398479
37.788668 -122.400558
37.787092 -122.399972
40.7311317 -73.9919311
51.500152 -0.126236
37.437328 -122.159928
<NA> <NA>
Getting the geo data for 201 Third Street, 2nd Floor, San Francisco
37.779507 -122.39071
37.7911148 -122.3954751
37.786942 -122.401245
40.751816 -73.971655
37.762125 -122.4147
37.789488 -122.397593
40.753385 -73.98962
37.766909 -122.406676
37.09024 -95.712891
37.785647 -122.405265
40.7445343 -73.9874008
40.7244 -73.99732
40.72948 -74.000195
37.768708 -122.402866
40.745216 -73.982807
37.765125 -122.4044034
37.783898 -122.395234
37.856331 -122.495222
40.781159 -73

In [121]:
len(all_companies_cleaned_df)

252

## 6. Cleaning San Francisco Companies as there are some wrong latitudes and longitudes

In [171]:
sf_companies_df = pd.read_csv('../data/san_francisco_companies.csv')

In [181]:
# Identify incorrect coordinates by converting them to strings and checking the prefix
incorrect_latitudes = sf_companies_df['latitude'].astype(str).str.startswith('37') == False
incorrect_longitudes = sf_companies_df['longitude'].astype(str).str.startswith('-122') == False

# Get the indices of the rows with incorrect coordinates
incorrect_coords_indices = sf_companies_df[incorrect_latitudes | incorrect_longitudes].index

# Display the rows with incorrect coordinates
incorrect_coords_df = sf_companies_df.loc[incorrect_coords_indices]
incorrect_coords_df[['name', 'address','latitude', 'longitude']] 

,name,address,latitude,longitude


In [183]:
sf_companies_df.at[103, 'latitude'] = 37.7820311
sf_companies_df.at[103, 'longitude'] = -122.3957106

In [184]:
sf_companies_df.at[89, 'latitude'] = 37.8017812
sf_companies_df.at[89, 'longitude'] = -122.4040438

In [185]:
sf_companies_df.at[65, 'latitude'] = 37.771696
sf_companies_df.at[65, 'longitude'] = -122.4025698

In [186]:
sf_companies_df.at[43, 'latitude'] = 37.7902397
sf_companies_df.at[43, 'longitude'] = -122.4027193

In [187]:
sf_companies_df.at[16, 'latitude'] = 37.7919443
sf_companies_df.at[16, 'longitude'] = -122.4026277

In [191]:
sf_companies_df

,name,category_code,number_of_employees,total_money_raised,country_code,city,address,zip_code,latitude,longitude,Company_Type
0,hi5,games_video,100.0,$52M,USA,San Francisco,55 Second Street,94105,37.788668,-122.400558,Pre-selected
1,Zynga,games_video,115.0,$860M,USA,San Francisco,365 Vermont St.,94103,37.765158,-122.404234,Pre-selected
2,iWin,games_video,103.0,$0,USA,San Francisco,45 Fremont Street,94105,37.781689,-122.391061,Pre-selected
3,Digg,news,NaN,$45M,USA,San Francisco,135 Mississippi St,94107,37.764726,-122.394523,Raised > $1M
4,Prosper,finance,NaN,$120M,USA,San Francisco,101 2nd Street,94105,37.789760,-122.402524,Raised > $1M
...,...,...,...,...,...,...,...,...,...,...,...
129,Recurrent Energy,cleantech,NaN,$85M,USA,San Francisco,300 California St.,94104,37.805289,-122.404448,Raised > $1M
130,Siluria Technologies,nanotech,NaN,$103M,USA,San Francisco,409 Illinois Street,94158,37.448491,-122.180281,Raised > $1M
131,ZoomSystems,hardware,NaN,$87M,USA,San Francisco,"22 4th Street, Floor 16",94103,37.781689,-122.391061,Raised > $1M
132,TeachScape,software,NaN,$16M,USA,San Francisco,71 Stevenson St.,94105,37.786905,-122.404392,Raised > $1M


In [189]:
# Check for duplicates based on the 'name' column only and keep the first occurrence
sf_cleaned_df = sf_companies_df.drop_duplicates(subset=['name'], keep='first')

In [190]:
sf_cleaned_df

,name,category_code,number_of_employees,total_money_raised,country_code,city,address,zip_code,latitude,longitude,Company_Type
0,hi5,games_video,100.0,$52M,USA,San Francisco,55 Second Street,94105,37.788668,-122.400558,Pre-selected
1,Zynga,games_video,115.0,$860M,USA,San Francisco,365 Vermont St.,94103,37.765158,-122.404234,Pre-selected
2,iWin,games_video,103.0,$0,USA,San Francisco,45 Fremont Street,94105,37.781689,-122.391061,Pre-selected
3,Digg,news,NaN,$45M,USA,San Francisco,135 Mississippi St,94107,37.764726,-122.394523,Raised > $1M
4,Prosper,finance,NaN,$120M,USA,San Francisco,101 2nd Street,94105,37.789760,-122.402524,Raised > $1M
...,...,...,...,...,...,...,...,...,...,...,...
129,Recurrent Energy,cleantech,NaN,$85M,USA,San Francisco,300 California St.,94104,37.805289,-122.404448,Raised > $1M
130,Siluria Technologies,nanotech,NaN,$103M,USA,San Francisco,409 Illinois Street,94158,37.448491,-122.180281,Raised > $1M
131,ZoomSystems,hardware,NaN,$87M,USA,San Francisco,"22 4th Street, Floor 16",94103,37.781689,-122.391061,Raised > $1M
132,TeachScape,software,NaN,$16M,USA,San Francisco,71 Stevenson St.,94105,37.786905,-122.404392,Raised > $1M


In [192]:
sf_cleaned_df.to_csv('../data/san_francisco_companies_cleaned_df.csv', index=False)